In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from PIL import Image
from six.moves import range


In [2]:
pickle_file= '/input/banglaIsolated_clean.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset_clean']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset_clean']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28) (139000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [3]:
image_size=28
num_labels=84
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28, 1) (139000, 84)
validation set (10000, 28, 28, 1) (10000, 84)
test set (10000, 28, 28, 1) (10000, 84)


In [4]:
#returns accuracy of model
def accuracy(target,predictions):
    return(100.0*np.sum(np.argmax(target,1) == np.argmax(predictions,1))/target.shape[0])

In [5]:
#need to batch the test data because running low on memory
class test_batchs:
    def __init__(self,data):
        self.data = data
        self.batch_index = 0
    def nextBatch(self,batch_size):
        if (batch_size+self.batch_index) > self.data.shape[0]:
            print ("batch sized is messed up")
        batch = self.data[self.batch_index:(self.batch_index+batch_size),:,:,:]
        self.batch_index= self.batch_index+batch_size
        return batch

#set the test batchsize
test_batch_size = 100

In [6]:
#use os to get our current working directory so we can save variable
file_path = os.getcwd()+'/model.ckpt'

In [9]:
def deeper_inception_conv_net():
    
    batch_size = 50
    patch_size1 = 3
    patch_size2 = 5
    depth = 16
    depth1 = 32
    depth2 = 16
    depth3 = 8
    concat_depth = 72 #concat_depth = depth2+depth3+depth3+depth2
    num_hidden = 1024
    num_hidden2 = 512
    keep_prob = 0.5
    decay_step = 2000
    base = 0.9

    graph = tf.Graph()

    with graph.as_default():
        # Input data.
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels),name="tf_train_dataset")
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels),name="tf_train_labels")
        tf_valid_dataset = tf.constant(valid_dataset,name="tf_valid_dataset")
        tf_test_dataset = tf.constant(test_dataset,name="tf_test_dataset")
        ##input for inference
        tf_input=tf.placeholder(tf.float32, shape=(1, image_size, image_size, num_channels),name="tf_input")
        ##Output for inference
        #output=tf.placeholder(tf.float32, shape=(1, num_labels),name="output")
        # Variables.
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, num_channels, depth], stddev=0.3),name="layer1_weights")
        layer1_biases = tf.Variable(tf.zeros([depth]),name="layer1_biases")
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, depth, depth1], stddev=0.05),name="layer2_weights")
        layer2_biases = tf.Variable(tf.constant(0.0, shape=[depth1]),name="layer2_biases")
        layer3_weights = tf.Variable(tf.truncated_normal(
          [((image_size + 3) // 4) * ((image_size + 3) // 4) * concat_depth, num_hidden], stddev=0.05),name="layer3_weights")
        layer3_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden]),name="layer3_biases")
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_hidden2], stddev=0.01),name="layer4_weights")
        layer4_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden2]),name="layer4_biases")
        layer5_weights = tf.Variable(tf.truncated_normal(
          [num_hidden2, num_labels], stddev=0.01),name="layer5_weights")
        layer5_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]),name="layer5_biases")

        inception1x1_weights = tf.Variable(tf.truncated_normal(
          [1, 1, depth1, depth2], stddev=0.25),name="inception1x1_weights")
        inception1x1_biases = tf.Variable(tf.constant(0.0, shape=[depth2]),name="inception1x1_biases")
        inception3x3_weights = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, depth2, depth3], stddev=0.05),name="inception3x3_weights")
        inception3x3_biases = tf.Variable(tf.constant(0.0, shape=[depth3]),name="inception3x3_biases")
        inception5x5_weights = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, depth2, depth3], stddev=0.08),name="inception5x5_weights")
        inception5x5_biases = tf.Variable(tf.constant(0.0, shape=[depth3]),name="inception5x5_biases")

        inception1x1_post_mxpool_wts = tf.Variable(tf.truncated_normal(
          [1, 1, depth1, depth2], stddev=0.04),name="inception1x1_post_mxpool_wts")
        post_maxpool_biases = tf.Variable(tf.constant(0.0, shape=[depth2]),name="post_maxpool_biases")


        ##For inception module2
        inception1x1_weights_mod2 = tf.Variable(tf.truncated_normal(
          [1, 1, 48, 24], stddev=0.25),name="inception1x1_weights_mod2")
        inception1x1_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[24]),name="inception1x1_biases_mod2")
        inception3x3_weights_mod2 = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, 24, 12], stddev=0.05),name="inception3x3_weights_mod2")
        inception3x3_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[12]),name="inception3x3_biases_mod2")
        inception5x5_weights_mod2 = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, 24, 12], stddev=0.08),name="inception5x5_weights_mod2")
        inception5x5_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[12]),name="inception5x5_biases_mod2")

        inception1x1_post_mxpool_wts_mod2 = tf.Variable(tf.truncated_normal(
          [1, 1, 48, 24], stddev=0.04),name="inception1x1_post_mxpool_wts_mod2")
        post_maxpool_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[24]),name="post_maxpool_biases_mod2")
        #End of inception module2

        global_step = tf.Variable(0, trainable = False)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(0.005, global_step, decay_step, base)
        
        
        # Model.
        def model(data, useDropout):
            print(data)
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            print(conv)
            max_pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
            print(max_pooled)
            hidden = tf.nn.relu(max_pooled + layer1_biases)
            print(hidden)
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
            print(conv)
            max_pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 1, 1, 1], padding='SAME')
            print(max_pooled)
            hidden = tf.nn.relu(max_pooled + layer2_biases)
            print(hidden)
            print("b4 inception")
            print(inception1x1_weights)
            print(inception3x3_weights)
            print(inception5x5_weights)
            inception1x1_conv = tf.nn.conv2d(hidden, inception1x1_weights, [1, 1, 1, 1], padding='SAME')
            inception1x1_relu = tf.nn.relu(inception1x1_conv + inception1x1_biases)

            inception3x3_conv = tf.nn.conv2d(inception1x1_relu, inception3x3_weights, [1, 1, 1, 1], padding='SAME')
            inception3x3_relu = tf.nn.relu(inception3x3_conv + inception3x3_biases)

            inception5x5_conv = tf.nn.conv2d(inception1x1_relu, inception5x5_weights, [1, 1, 1, 1], padding='SAME')
            inception5x5_relu = tf.nn.relu(inception5x5_conv + inception5x5_biases)

            inception3x3_maxpool = tf.nn.max_pool(hidden, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
            inception1x1_post_maxpool = tf.nn.conv2d(inception3x3_maxpool, inception1x1_post_mxpool_wts, [1, 1, 1, 1], padding='SAME')
            inception1x1_post_maxpool = tf.nn.relu(inception1x1_post_maxpool + post_maxpool_biases)
            print(inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool)
            concat_filter = tf.concat([inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool],3)
            print("after inception")
            print(concat_filter)

            print("b4 inception 2")

            inception1x1_conv = tf.nn.conv2d(concat_filter, inception1x1_weights_mod2, [1, 1, 1, 1], padding='SAME')
            inception1x1_relu = tf.nn.relu(inception1x1_conv + inception1x1_biases_mod2)

            inception3x3_conv = tf.nn.conv2d(inception1x1_relu, inception3x3_weights_mod2, [1, 1, 1, 1], padding='SAME')
            inception3x3_relu = tf.nn.relu(inception3x3_conv + inception3x3_biases_mod2)

            inception5x5_conv = tf.nn.conv2d(inception1x1_relu, inception5x5_weights_mod2, [1, 1, 1, 1], padding='SAME')
            inception5x5_relu = tf.nn.relu(inception5x5_conv + inception5x5_biases_mod2)

            inception3x3_maxpool = tf.nn.max_pool(concat_filter, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
            inception1x1_post_maxpool = tf.nn.conv2d(inception3x3_maxpool, inception1x1_post_mxpool_wts_mod2, [1, 1, 1, 1], padding='SAME')
            inception1x1_post_maxpool = tf.nn.relu(inception1x1_post_maxpool + post_maxpool_biases_mod2)
            print(inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool)
            concat_filter = tf.concat([inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool],3)
            print("after inception 2")
            print(concat_filter)

            concat_maxpooled = tf.nn.max_pool(concat_filter, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
            print(concat_maxpooled)
            shape = concat_maxpooled.get_shape().as_list()

            reshape = tf.reshape(concat_maxpooled, [shape[0], shape[1] * shape[2] * shape[3]])
            print(reshape)
            if useDropout == 1:
                dropout_layer2 = tf.nn.dropout(tf.nn.relu(reshape), keep_prob)
            else:
                dropout_layer2 = tf.nn.relu(reshape)
            print(dropout_layer2)
            print("1st fc")
            hidden = tf.nn.relu(tf.matmul(dropout_layer2, layer3_weights) + layer3_biases)
            print(hidden)
            print("2nd fc")
            hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
            print(hidden)
            print("after fully Connected")
            return tf.matmul(hidden, layer5_weights) + layer5_biases

        # Training computation.
        logits = model(tf_train_dataset, 1)
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels),name="loss")

        # Optimizer.
        optimizer = tf.train.AdamOptimizer(0.001).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(model(tf_train_dataset, 0),name="train_prediction")
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 0),name="valid_prediction")
        test_prediction = tf.nn.softmax(model(tf_test_dataset, 0),name="test_prediction")
        output=tf.nn.softmax(model(tf_input,0),name="output")
        
        saver = tf.train.Saver()
        
    num_steps = 30001

    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 100 == 0):
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                #print(tf.Print(layer1_weights, [layer1_weights]).eval())
                model_path=saver.save(session,'/output/banglaInceptionM2.ckpt', global_step=step)
                print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)) 
        print('Model saved in: %s' % model_path)
deeper_inception_conv_net()


Tensor("tf_train_dataset:0", shape=(50, 28, 28, 1), dtype=float32)
Tensor("Conv2D:0", shape=(50, 28, 28, 16), dtype=float32)
Tensor("MaxPool:0", shape=(50, 14, 14, 16), dtype=float32)
Tensor("Relu:0", shape=(50, 14, 14, 16), dtype=float32)
Tensor("Conv2D_1:0", shape=(50, 14, 14, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(50, 14, 14, 32), dtype=float32)
Tensor("Relu_1:0", shape=(50, 14, 14, 32), dtype=float32)
b4 inception
Tensor("inception1x1_weights/read:0", shape=(1, 1, 32, 16), dtype=float32)
Tensor("inception3x3_weights/read:0", shape=(3, 3, 16, 8), dtype=float32)
Tensor("inception5x5_weights/read:0", shape=(5, 5, 16, 8), dtype=float32)
Tensor("Relu_2:0", shape=(50, 14, 14, 16), dtype=float32) Tensor("Relu_3:0", shape=(50, 14, 14, 8), dtype=float32) Tensor("Relu_4:0", shape=(50, 14, 14, 8), dtype=float32) Tensor("Relu_5:0", shape=(50, 14, 14, 16), dtype=float32)
after inception
Tensor("concat:0", shape=(50, 14, 14, 48), dtype=float32)
b4 inception 2
Tensor("Relu_6:0", shape=(

Initialized
Minibatch loss at step 0: 4.431573
Minibatch accuracy: 2.0%
Validation accuracy: 1.2%
Minibatch loss at step 100: 3.659979
Minibatch accuracy: 6.0%
Validation accuracy: 12.0%
Minibatch loss at step 200: 2.981443
Minibatch accuracy: 24.0%
Validation accuracy: 27.7%
Minibatch loss at step 300: 2.462240
Minibatch accuracy: 40.0%
Validation accuracy: 38.1%
Minibatch loss at step 400: 1.905987
Minibatch accuracy: 48.0%
Validation accuracy: 50.1%
Minibatch loss at step 500: 1.525109
Minibatch accuracy: 64.0%
Validation accuracy: 57.2%
Minibatch loss at step 600: 1.519067
Minibatch accuracy: 64.0%
Validation accuracy: 60.9%
Minibatch loss at step 700: 1.212067
Minibatch accuracy: 64.0%
Validation accuracy: 63.8%
Minibatch loss at step 800: 1.222964
Minibatch accuracy: 70.0%
Validation accuracy: 68.1%
Minibatch loss at step 900: 0.924427
Minibatch accuracy: 68.0%
Validation accuracy: 70.2%
Minibatch loss at step 1000: 0.894234
Minibatch accuracy: 78.0%
Validation accuracy: 72.3%
Mi

Validation accuracy: 90.9%
Minibatch loss at step 9100: 0.424912
Minibatch accuracy: 90.0%
Validation accuracy: 91.0%
Minibatch loss at step 9200: 0.481834
Minibatch accuracy: 88.0%
Validation accuracy: 91.4%
Minibatch loss at step 9300: 0.452290
Minibatch accuracy: 84.0%
Validation accuracy: 91.1%
Minibatch loss at step 9400: 0.484656
Minibatch accuracy: 92.0%
Validation accuracy: 91.4%
Minibatch loss at step 9500: 0.208677
Minibatch accuracy: 92.0%
Validation accuracy: 90.5%
Minibatch loss at step 9600: 0.274161
Minibatch accuracy: 94.0%
Validation accuracy: 90.9%
Minibatch loss at step 9700: 0.777506
Minibatch accuracy: 84.0%
Validation accuracy: 90.6%
Minibatch loss at step 9800: 0.700114
Minibatch accuracy: 84.0%
Validation accuracy: 91.4%
Minibatch loss at step 9900: 0.306652
Minibatch accuracy: 92.0%
Validation accuracy: 90.7%
Minibatch loss at step 10000: 0.300734
Minibatch accuracy: 86.0%
Validation accuracy: 91.4%
Minibatch loss at step 10100: 0.385585
Minibatch accuracy: 88.

Minibatch loss at step 18000: 0.393656
Minibatch accuracy: 96.0%
Validation accuracy: 92.8%
Minibatch loss at step 18100: 0.095050
Minibatch accuracy: 96.0%
Validation accuracy: 93.2%
Minibatch loss at step 18200: 0.449810
Minibatch accuracy: 90.0%
Validation accuracy: 93.2%
Minibatch loss at step 18300: 0.283647
Minibatch accuracy: 94.0%
Validation accuracy: 92.9%
Minibatch loss at step 18400: 0.215227
Minibatch accuracy: 94.0%
Validation accuracy: 93.2%
Minibatch loss at step 18500: 0.294160
Minibatch accuracy: 92.0%
Validation accuracy: 92.3%
Minibatch loss at step 18600: 0.394300
Minibatch accuracy: 90.0%
Validation accuracy: 93.3%
Minibatch loss at step 18700: 0.260480
Minibatch accuracy: 94.0%
Validation accuracy: 93.0%
Minibatch loss at step 18800: 0.479051
Minibatch accuracy: 94.0%
Validation accuracy: 93.0%
Minibatch loss at step 18900: 0.383618
Minibatch accuracy: 94.0%
Validation accuracy: 93.1%
Minibatch loss at step 19000: 0.441242
Minibatch accuracy: 90.0%
Validation accu

Minibatch loss at step 26900: 0.693166
Minibatch accuracy: 86.0%
Validation accuracy: 94.9%
Minibatch loss at step 27000: 0.499260
Minibatch accuracy: 94.0%
Validation accuracy: 94.8%
Minibatch loss at step 27100: 0.390144
Minibatch accuracy: 88.0%
Validation accuracy: 94.7%
Minibatch loss at step 27200: 0.303716
Minibatch accuracy: 94.0%
Validation accuracy: 94.7%
Minibatch loss at step 27300: 0.265748
Minibatch accuracy: 94.0%
Validation accuracy: 94.7%
Minibatch loss at step 27400: 0.344153
Minibatch accuracy: 94.0%
Validation accuracy: 94.9%
Minibatch loss at step 27500: 0.161504
Minibatch accuracy: 100.0%
Validation accuracy: 94.8%
Minibatch loss at step 27600: 0.160210
Minibatch accuracy: 96.0%
Validation accuracy: 94.9%
Minibatch loss at step 27700: 0.158438
Minibatch accuracy: 98.0%
Validation accuracy: 94.8%
Minibatch loss at step 27800: 0.345999
Minibatch accuracy: 94.0%
Validation accuracy: 94.7%
Minibatch loss at step 27900: 0.254815
Minibatch accuracy: 96.0%
Validation acc